In [1]:
import sys
import os
sys.path.append('/home/chris/Documents/code/pyProjects/cbstools-public/python')
import cbstools as cbs
from os.path import join, exists
import nibabel as nb
import numpy as np
import glob as glob
import pandas as pd
cbs=reload(cbs)

## this version uses MD input as a sigmoid filtered input of type "Filters"

In [2]:
FA_files = sorted(glob.glob('/home/chris/Documents/projects_working/FAMD_seg/source/FA_MD_mipav_axial/*FA*.gz'))
MD_files = sorted(glob.glob('/home/chris/Documents/projects_working/FAMD_seg/source/FA_MD_mipav_axial/*MD*.gz'))
t1w_files = sorted(glob.glob('/home/chris/Documents/projects_working/FAMD_seg/source/t1w_mipav_axial/*t1*.gz'))
t1w_seg_files = sorted(glob.glob('/home/chris/Documents/projects_working/FAMD_seg/processing/t1w_seg/*seg.nii.gz'))
out_dir='/home/chris/Documents/projects_working/FAMD_seg/processing/test2'
out_dir_bal = '/home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal'
checking_dir = '/home/chris/Documents/projects_working/FAMD_seg/processing/tmp'
atlas_L_R_pairable_structures = np.array([[11, 12],[17,18],[26,27],[30,31],[32,33],
                                          [34,35],[36,37],[38,39],[40,41],[47,48]]) #L/R structures for prior est

In [3]:
# ensure that the ordering is correct
for idx,t1w_seg_file in enumerate(t1w_seg_files):
    print(t1w_seg_file).split("/")[-1]
    print(FA_files[idx].split("/")[-1])
    print(MD_files[idx].split("/")[-1])
    print ""

AD2T090602_mr1095_t1_pl_clone_transform_N3Corrected_seg.nii.gz
AD2T090602__FA_clone_transform.nii.gz
AD2T090602__MD_clone_transform.nii.gz

HN3T090602_mr1260_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HN3T090602__FA_clone_transform.nii.gz
HN3T090602__MD_clone_transform.nii.gz

HSLT090615_mr1088_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HSLT090615__FA_clone_transform.nii.gz
HSLT090615__MD_clone_transform.nii.gz

HSNT090625_mr1236_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HSNT090625__FA_clone_transform.nii.gz
HSNT090625__MD_clone_transform.nii.gz

HT5T090612_mr1239_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HT5T090612__FA_clone_transform.nii.gz
HT5T090612__MD_clone_transform.nii.gz

KK6T090706_mr1040_t1_pl_clone_transform_N3Corrected_seg.nii.gz
KK6T090706__FA_clone_transform.nii.gz
KK6T090706__MD_clone_transform.nii.gz

KKCT090615_mr1261_t1_pl_clone_transform_N3Corrected_seg.nii.gz
KKCT090615__FA_clone_transform.nii.gz
KKCT090615__MD_clone_transform.nii.gz

NM3T090701_mr1168_t1

In [4]:
## deprecated!!
#combine FA and MD into list of lists, you should do this differently
#FA_MD = []
#for idx in range(0,len(FA_files)):
#    FA_MD.append([FA_files[idx],MD_files[idx]])

In [5]:
cbs.get_MGDM_seg_contrast_names(os.path.join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS))

['DWIFA3T',
 'DWIMD3T',
 'T1map9T',
 'Mp2rage9T',
 'T1map7T',
 'Mp2rage7T',
 'PV',
 'Filters',
 'T1pv',
 'Mprage3T',
 'T1map3T',
 'Mp2rage3T',
 'HCPT1w',
 'HCPT2w',
 'NormMPRAGE']

In [6]:
# check overlap between segs
cbs=reload(cbs)
seg1='/home/chris/Documents/projects_working/FAMD_seg/processing/test2/AD2T090602__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters.nii.gz'
seg2='/home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal/AD2T090602__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz'
lut=cbs.compare_atlas_segs_priors(seg1,seg2,atlas_file_orig = join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS),
                              metric_contrast_name="DWIFA3T")

Structure index: 50, Dura
Dice similarity: 0.864956229171
Structure index: 8, Arteries
Dice similarity: 0.0
Structure index: 10, Sulcal-CSF
Dice similarity: 0.910131435019
Structure index: 11, VentricleL
Dice similarity: 0.951573227595
Structure index: 12, VentricleR
Dice similarity: 0.928616550853
Structure index: 13, Ventricle3
Dice similarity: 0.780269058296
Structure index: 14, Ventricle4
Dice similarity: 0.864448051948
Structure index: 17, ChoroidPlexusL
Dice similarity: 0.0
Structure index: 18, ChoroidPlexusR
Dice similarity: 0.0
Structure index: 20, Sinuses
Dice similarity: 0.75601750547
Structure index: 25, Cerebellum-GM
Dice similarity: 0.929467074158
Structure index: 26, Cerebrum-GML
Dice similarity: 0.95795353334
Structure index: 27, Cerebrum-GMR
Dice similarity: 0.901934599822
Structure index: 30, AmygdalaL
Dice similarity: 0.83032711709
Structure index: 31, AmygdalaR
Dice similarity: 0.762857142857
Structure index: 32, CaudateL
Dice similarity: 0.929463820927
Structure ind

In [10]:
# extract initial priors, to see what they would look like
cbs=reload(cbs)
metric_contrast_name="DWIFA3T"
atlas_file = join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS)
new_atlas = join(cbs.ATLAS_DIR,"xxx_atlas_cjs_test_FA.txt")
medians, spreads = cbs.generate_group_intensity_priors(orig_seg_files=t1w_seg_files,
                                                       metric_files=FA_files,
                                                       metric_contrast_name=metric_contrast_name,
                                                       min_quart_diff=0.05,
                                                       seg_null_value=0, background_idx=1,
                                                       atlas_file = atlas_file,
                                                       erosion_iterations=0,intermediate_output_dir=checking_dir)


seg_idxs = medians[0,:]
grp_median = np.median(medians[1:,:],axis=0)
grp_spread = np.median(spreads[1:,:],axis=0)
new_atlas_file=cbs.write_priors_to_atlas(grp_median,grp_spread,join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS),new_atlas,metric_contrast_name)

AD2T090602_mr1095_t1_pl_clone_transform_N3Corrected_seg.nii.gz
AD2T090602__FA_clone_transform.nii.gz

HN3T090602_mr1260_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HN3T090602__FA_clone_transform.nii.gz

HSLT090615_mr1088_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HSLT090615__FA_clone_transform.nii.gz

HSNT090625_mr1236_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HSNT090625__FA_clone_transform.nii.gz

HT5T090612_mr1239_t1_pl_clone_transform_N3Corrected_seg.nii.gz
HT5T090612__FA_clone_transform.nii.gz

KK6T090706_mr1040_t1_pl_clone_transform_N3Corrected_seg.nii.gz
KK6T090706__FA_clone_transform.nii.gz

KKCT090615_mr1261_t1_pl_clone_transform_N3Corrected_seg.nii.gz
KKCT090615__FA_clone_transform.nii.gz

NM3T090701_mr1168_t1_pl_clone_transform_N3Corrected_seg.nii.gz
NM3T090701__FA_clone_transform.nii.gz

RT5T090709_mr983_t1_pl_clone_transform_N3Corrected_seg.nii.gz
RT5T090709__FA_clone_transform.nii.gz

SF3T090612_mr1139_t1_pl_clone_transform_N3Corrected_seg.nii.gz
SF3T090612__FA_clone

In [11]:
cbs=reload(cbs)
# create a new set of medians/spreads for balanced L and R hemisphere by taking the max
# uses the manually set pairable_structures
new_atlas_balanced = join(cbs.ATLAS_DIR,"xxx_atlas_cjs_test_FA_LR_balanced.txt")
medians_L_R_balanced = np.copy(medians)
spreads_L_R_balanced = np.copy(spreads)

for pair in atlas_L_R_pairable_structures:
    p0=medians_L_R_balanced[1:,seg_idxs==pair[0]]
    p1=medians_L_R_balanced[1:,seg_idxs==pair[1]]
    medians_L_R_balanced[1:,seg_idxs==pair[0]] = np.max([p0,p1],axis = 0)
    medians_L_R_balanced[1:,seg_idxs==pair[1]] = np.max([p0,p1],axis = 0)
    
    p0=spreads_L_R_balanced[1:,seg_idxs==pair[0]]
    p1=spreads_L_R_balanced[1:,seg_idxs==pair[1]]
    spreads_L_R_balanced[1:,seg_idxs==pair[0]] = np.max([p0,p1],axis = 0)
    spreads_L_R_balanced[1:,seg_idxs==pair[1]] = np.max([p0,p1],axis = 0)

grp_median_balanced = np.median(medians_L_R_balanced[1:,:],axis=0)
grp_spread_balanced = np.median(spreads_L_R_balanced[1:,:],axis=0)
new_atlas_file_balanced=cbs.write_priors_to_atlas(grp_median_balanced,grp_spread_balanced,join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS),new_atlas_balanced,metric_contrast_name)

New atlas file written to: 
/home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA_LR_balanced.txt


In [7]:
#before extracting from the MD images, we need to fit to a sinusoid to extablish the tissue/CSF boundary
cbs=reload(cbs)
MD_filt_dir = '/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/'
MD_filt_files = []
for MD_file in MD_files:
    out_file = join(MD_filt_dir,MD_file.split("/")[-1].split(".")[0]+"_filt.nii.gz")
    print(out_file)
    if not exists(out_file):
        cbs.filter_sigmoid(MD_file,output_fname=out_file)
    MD_filt_files.append(out_file)

/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/AD2T090602__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/HN3T090602__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/HSLT090615__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/HSNT090625__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/HT5T090612__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/KK6T090706__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/KKCT090615__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/NM3T090701__MD_clone_transform_filt.nii.gz
/home/chris/Documents/projects_working/FAMD_seg/processing/MD_filt/RT5T090709__MD_clone_transform_filt.nii.gz
/home/chri

In [8]:
# now we can perform a segmentation with our newly created priors!
cbs.setup_JVM()
new_new_atlas_file = join(cbs.ATLAS_DIR,"xxx_atlas_cjs_test_FA.txt")
cbs=reload(cbs)
d2,a2,h2=cbs.MGDMBrainSegmentation_v2(con1_files=FA_files,con1_type='DWIFA3T',
                                   con2_files=MD_filt_files,con2_type='Filters',
                                   output_dir=out_dir,atlas_file=new_new_atlas_file, num_steps=5, topology='wcs',
                                   topology_lut_dir=None, file_suffix = "_new_atlas_ero0_min0p05_FA_Filters")

Java virtual machine successfully started.
Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
DWIFA3T:	AD2T090602__FA_clone_transform.nii.gz
Filters:	AD2T090602__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/projects_working/FAMD_seg/processing/test2/AD2T090602__FA_clone_transform_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test2
Execution completed
Input files and filetypes:
DWIFA3T:	HN3T090602__FA_clone_transform.nii.gz
Filters:	HN3T090602__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Do

In [44]:
# segmentation with the LR balanced priors
cbs=reload(cbs)
cbs.setup_JVM()
d2,a2,h2=cbs.MGDMBrainSegmentation_v2(con1_files=FA_files,con1_type='DWIFA3T',
                                      con2_files=MD_filt_files,con2_type='Filters',
                                      output_dir=out_dir_bal,atlas_file=new_atlas_balanced, 
                                      num_steps=5, topology='wcs',topology_lut_dir=None, 
                                      file_suffix = "_new_atlas_ero0_min0p05_FA_Filters_LR_balanced")

Java virtual machine successfully started.
Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA_LR_balanced.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
DWIFA3T:	HSNT090625__FA_clone_transform.nii.gz
Filters:	HSNT090625__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal

Execution completed
Input files and filetypes:
DWIFA3T:	HT5T090612__FA_clone_transform.nii.gz
Filters:	HT5T090612__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal

Execution completed
In

In [9]:
## 2nd level, now extract based on the newly segmented regions
# extract initial priors, to see what they would look like
FA_seg_files_balanced = sorted(glob.glob('/home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal/*seg*'))

cbs=reload(cbs)
metric_contrast_name="DWIFA3T"
atlas_file = join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS)
new_atlas_v2 = "/home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA_LR_balanced.txt"
new_atlas_out = "/home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA_LR_balanced_v2.txt"
medians, spreads = cbs.generate_group_intensity_priors(orig_seg_files=FA_seg_files_balanced,
                                                       metric_files=FA_files,
                                                       metric_contrast_name=metric_contrast_name,
                                                       min_quart_diff=0.05,
                                                       seg_null_value=0, background_idx=1,
                                                       atlas_file = new_atlas_v2,
                                                       erosion_iterations=0,intermediate_output_dir=None)



seg_idxs = medians[0,:]
#grp_median = np.median(medians[1:,:],axis=0)
#grp_spread = np.median(spreads[1:,:],axis=0)
#new_atlas_file=cbs.write_priors_to_atlas(grp_median,grp_spread,join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS),new_atlas_out,metric_contrast_name)

# create a new set of medians/spreads for balanced L and R hemisphere by taking the max
# uses the manually set pairable_structures
medians_L_R_balanced = np.copy(medians)
spreads_L_R_balanced = np.copy(spreads)

for pair in atlas_L_R_pairable_structures:
    p0=medians_L_R_balanced[1:,seg_idxs==pair[0]]
    p1=medians_L_R_balanced[1:,seg_idxs==pair[1]]
    medians_L_R_balanced[1:,seg_idxs==pair[0]] = np.max([p0,p1],axis = 0)
    medians_L_R_balanced[1:,seg_idxs==pair[1]] = np.max([p0,p1],axis = 0)
    
    p0=spreads_L_R_balanced[1:,seg_idxs==pair[0]]
    p1=spreads_L_R_balanced[1:,seg_idxs==pair[1]]
    spreads_L_R_balanced[1:,seg_idxs==pair[0]] = np.max([p0,p1],axis = 0)
    spreads_L_R_balanced[1:,seg_idxs==pair[1]] = np.max([p0,p1],axis = 0)

grp_median_balanced = np.median(medians_L_R_balanced[1:,:],axis=0)
grp_spread_balanced = np.median(spreads_L_R_balanced[1:,:],axis=0)
new_atlas_file_balanced=cbs.write_priors_to_atlas(grp_median_balanced,grp_spread_balanced,join(cbs.ATLAS_DIR,cbs.DEFAULT_ATLAS),new_atlas_out,metric_contrast_name)

AD2T090602__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
AD2T090602__FA_clone_transform.nii.gz

HN3T090602__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
HN3T090602__FA_clone_transform.nii.gz

HSLT090615__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
HSLT090615__FA_clone_transform.nii.gz

HSNT090625__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
HSNT090625__FA_clone_transform.nii.gz

HT5T090612__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
HT5T090612__FA_clone_transform.nii.gz

KK6T090706__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
KK6T090706__FA_clone_transform.nii.gz

KKCT090615__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
KKCT090615__FA_clone_transform.nii.gz

NM3T090701__FA_clone_transform_seg_new_atlas_ero0_min0p05_FA_Filters_LR_balanced.nii.gz
NM3T090701__FA_clone_tr

In [10]:
# segmentation with the LR balanced priors, v2
out_dir_bal_v2 = "/home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal_v2"
cbs=reload(cbs)
cbs.setup_JVM()
d2,a2,h2=cbs.MGDMBrainSegmentation_v2(con1_files=FA_files,con1_type='DWIFA3T',
                                      con2_files=MD_filt_files,con2_type='Filters',
                                      output_dir=out_dir_bal_v2,atlas_file=new_atlas_out, 
                                      num_steps=5, topology='wcs',topology_lut_dir=None, 
                                      file_suffix = "_new_atlas_ero0_min0p05_FA_Filters_LR_balanced_v2")

Java virtual machine successfully started.
Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/xxx_atlas_cjs_test_FA_LR_balanced_v2.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
DWIFA3T:	AD2T090602__FA_clone_transform.nii.gz
Filters:	AD2T090602__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal_v2

Execution completed
Input files and filetypes:
DWIFA3T:	HN3T090602__FA_clone_transform.nii.gz
Filters:	HN3T090602__MD_clone_transform_filt.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test2_bal_v2

Execution com

In [45]:
# segmentation with our old priors
cbs.setup_JVM()
cbs=reload(cbs)
d,a,h=cbs.MGDMBrainSegmentation_v2(con1_files=FA_files[0],con1_type='DWIFA3T',
                                   con2_files=MD_filt_files[0],con2_type='Filters',
                                   output_dir=out_dir,atlas_file=atlas_file,
                                   topology_lut_dir=None, file_suffix = "_orig_ero0")


Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  /home/chris/Documents/projects_working/FAMD_seg/source/FA_MD_mipav_axial/AD2T090602__FA_clone_transform.nii.gz
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/projects_working/FAMD_seg/processing/test1/AD2T090602__FA_clone_transform_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/projects_working/FAMD_seg/processing/test1
Execution completed
